In [2]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 25.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpContinuous, PULP_CBC_CMD
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from geopy.distance import geodesic
import sys

In [10]:
# Load data
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/CFLP/CFLP_infiles/hs_full_geo.geojson').set_index('pu_2324_84')
pu = pu.to_crs('EPSG:4326')

# Adjust demand for half SGR
pu['basez+gen'] = pu['basez'] + 0.15 * pu['student_gen']
d = pu['basez+gen'].to_dict()
I = list(d.keys())

In [18]:
# Load schools and determine existing sites
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/CFLP/CFLP_infiles/dps_hs_locations.geojson').to_crs('EPSG:4326')
schools['pu'] = None
for i, geometry in enumerate(pu['geometry']):
    in_geometry = geometry.contains(schools['geometry'])
    pu_id = pu.index[i]
    schools.loc[in_geometry, 'pu'] = pu_id

not_central = pu[pu['Region'] != 'Central']
J = list(not_central.index)
M = {idx: 1550 for idx in J}
M.update({45: 1400, 507: 1510, 602: 1340, 566: 1240, 290: 1335})
existing_sites = {602, 290, 45, 566, 507}

In [30]:
# Build distance matrix
centroids = pu.geometry.centroid
coords = {idx: (geom.y, geom.x) for idx, geom in centroids.items()}
c = {(i, j): geodesic(coords[i], coords[j]).miles for i in I for j in J}

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_89674/3756431071.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = pu.geometry.centroid


In [32]:
# Model
model = LpProblem("CFLP", LpMinimize)
x = {(i, j): LpVariable(f"x_{i}_{j}", lowBound=0, cat=LpContinuous) for i in I for j in J}
y = {j: LpVariable(f"y_{j}", cat=LpBinary) for j in J}
BIG_M = sum(d.values())

# Objective
model += lpSum(c[i, j] * x[i, j] for i in I for j in J)

# Constraints
for i in I:
    model += lpSum(x[i, j] for j in J) == d[i]

for j in J:
    model += lpSum(x[i, j] for i in I) <= 1.05 * M[j] * y[j]
    model += lpSum(x[i, j] for i in I) >= 0.7 * M[j] * y[j]

for i in I:
    for j in J:
        model += x[i, j] <= BIG_M * y[j]

for j in existing_sites:
    model += y[j] == 1

model += lpSum(y[j] for j in J) <= 6

In [34]:
# Solve
solver = PULP_CBC_CMD(msg=1)
model.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/61c9e5264099435ca671762930de0db7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/61c9e5264099435ca671762930de0db7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 629523 COLUMNS
At line 4395599 RHS
At line 5025118 BOUNDS
At line 5025856 ENDATA
Problem MODEL has 629518 rows, 627924 columns and 3138151 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 11835.3 - 3.01 seconds
Cgl0004I processed model has 507240 rows, 509262 columns (732 integer (732 of which binary)) and 2534496 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 4.05835e-06
Cbc0038I Pass   1: (51138.77 seco

1

In [ ]:
# Collect solution
assignments = {j: [] for j in J if y[j].varValue > 0.5}
for (i, j), var in x.items():
    if var.varValue > 1e-3:
        if y[j].varValue < 0.5:
            print(f"WARNING: PU {i} assigned to CLOSED facility {j}")
        assignments[j].append(i)

In [ ]:
# Student counts
student_counts = {
    j: sum(pu.loc[i, 'basez'] for i in pus)
    for j, pus in assignments.items()
}

In [ ]:
# Output
solution = {
    "objective_value": model.objective.value(),
    "facilities": list(assignments.keys()),
    "assignments": assignments,
    "student_count": student_counts
}

In [ ]:
# Export assignment map
pu_new = pu.copy()
pu_to_fac = {i: j for j, pus in assignments.items() for i in pus}
pu_new["assignment"] = pu.index.map(pu_to_fac)
pu_new.to_file("CFLP_hs_halfSGR_v2_pulp.geojson", driver="GeoJSON")